In [1]:
from functions import get_data , calculate_sma



In [2]:
using_data=get_data('GOLD')




In [3]:
import pandas as pd

In [4]:
def body_size(openp,close):
    bs=abs(close-openp)
    return bs

In [5]:
def calc_vs(openp,close,low,high):
    candle_spread=high-low
    body_size=abs(close-openp)
    ratio=candle_spread/body_size
    return ratio
    

In [6]:
copy1=using_data

In [7]:
copy1['body_size'] = body_size(copy1['Open'], copy1['Close'])

# Find the indices where body size is 0
zero_body_size_indices = copy1[copy1['body_size'] == 0].index

# Print the indices
print("Indices where body size is 0:", zero_body_size_indices)
print('number of values with zero are:',len(zero_body_size_indices))

Indices where body size is 0: Index([    36,    140,    170,    437,    527,    761,    915,   1042,   1068,
         1266,
       ...
       208580, 209122, 209169, 209398, 209489, 209723, 209889, 210017, 210065,
       210272],
      dtype='int64', length=1544)
number of values with zero are: 1544


In [8]:
print('the dataset is this long: ',len(copy1))
g=(len(zero_body_size_indices))/(len(copy1))
print('the chnces of you gettibg a trade using zero volume are :',g)

the dataset is this long:  210273
the chnces of you gettibg a trade using zero volume are : 0.0073428352665344575


In [9]:
import datetime

In [10]:
# copy1['time'] = pd.to_datetime(copy1['time'], unit='s')


In [11]:
copy1['ratio'] = calc_vs(copy1['Open'], copy1['Close'], copy1['Low'], copy1['High'])

# Create a histogram using Plotly


In [12]:
def find_candle_direction(open,close):
    if open>close:
        return 'red'
    else:
        return 'green'

In [13]:
from functions import calc_prof

In [14]:
def breakout(data,multiplier,sl_adjust):
    istrade=None
    entry_price=0
    p_sl=0
    condition1=0
    p_tp=0
    sl=0
    opendate=0
    closedate=0
    trade_type=''
    trades=[]
    tp=0
    for i in range(len(data)):
        if data['body_size'][i]==0:
            condition1=1
            p_sl=data['Low'][i]
            p_tp=data['High'][i]
            
        if condition1==1 and istrade==None:
            openp=data['Open'][i]
            close=data['Close'][i]
            candle_type=find_candle_direction(openp,close)
            if candle_type=='red':
                entry_price=close
                istrade=True
                trade_type='Sell'
                sl=p_tp+sl_adjust
                predicted_sl=sl-entry_price
                sup_tp=predicted_sl*multiplier
                tp=entry_price-sup_tp
                condition1=0
            else:
                entry_price=close
                istrade=True
                trade_type='Buy'
                opendate=data['time'][i]
                sl=p_sl-sl_adjust
                predicted_sl=entry_price-sl
                sup_tp=predicted_sl*multiplier
                tp=entry_price+sup_tp
                condition1=0
        if istrade==True:
            if trade_type=='Buy':
                if data['Close'][i]>tp:
                    exit=data['Close'][i]
                    profit=calc_prof(entry_price,exit)
                    closedate=data['time'][i]
                    trades.append(('Buy', opendate, closedate, profit))
                    istrade=None
                elif data['Close'][i]<sl:
                    exit=data['Close'][i]
                    profit=calc_prof(entry_price,exit)
                    closedate=data['time'][i]
                    trades.append(('Buy', opendate, closedate, profit))
                    istrade=None
            else:
                if data['Close'][i]<tp:
                    exit=data['Close'][i]
                    profit=calc_prof(entry_price,exit)
                    closedate=data['time'][i]
                    trades.append(('Sell', opendate, closedate, profit))
                    istrade=None
                elif data['Close'][i]>sl:
                    exit=data['Close'][i]
                    profit=calc_prof(entry_price,exit)
                    closedate=data['time'][i]
                    trades.append(('Sell', opendate, closedate, profit))
                    istrade=None
    return trades            
                
                
                

In [15]:
from functions import evaluate

In [16]:
test_sl_values = [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
multiplier_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]
evaluated_results = []

# Loop through parameter combinations
for sl_value in test_sl_values:
    for multiplier_value in multiplier_values:
        o_volume_breakout = breakout(copy1, multiplier_value, sl_value)
        evaluation_name = f'zero volume breakout SL added of {sl_value} and multiplier of {multiplier_value}'
        
        # Assuming evaluate returns a list with the details
        evaluated_info = evaluate(o_volume_breakout, evaluation_name)
        
        # Create a dictionary with the information and parameters
        result_dict = {
            'Strategy': 'Zero Volume Breakout',
            'SL_Value': sl_value,
            'Multiplier_Value': multiplier_value,
            'Total_Profit': evaluated_info[0],
            'Num_Buy_Trades': evaluated_info[1],
            'Total_Buy_Profit': evaluated_info[2],
            'Num_Sell_Trades': evaluated_info[3],
            'Total_Sell_Profit': evaluated_info[4],
            'Max_Profit_Index': evaluated_info[5],
            'Index_of_Max_Profit': evaluated_info[6],
            'Max_Duration': evaluated_info[7],
            'ID_Max_Duration': evaluated_info[8],
            'total_profitable_trades':evaluated_info[9],
            'total_unprofitable_trades':evaluated_info[10],
            'total_profitable_trades':evaluated_info[11],
            'total_loss_trades':evaluated_info[12]
        }
        
        # Append the dictionary to the list
        evaluated_results.append(result_dict)


In [ ]:
o_strat_results = pd.DataFrame(evaluated_results)
o_strat_results.to_csv('o_strat_results.csv', index=False)
